# Demo2: extract story from game log by using GPT-2 Summary

## Env setup

Install older TensorFlow version

Source code relies on older TensorFlow version. Installing TF v1.15 seems to fix the issue of *ModuleNotFoundError when training the model*. (Workaround found here: https://colab.research.google.com/notebooks/tensorflow_version.ipynb#scrollTo=8UvRkm1JGUrk) 

In [1]:
%tensorflow_version 1.x
!pip -q install tensorflow==1.15 && pip -q install tensorflow-gpu==1.15
!pip -q install 'tensorflow-estimator<1.15.0rc0,>=1.14.0rc0' --force-reinstall

TensorFlow 1.x selected.
     |████████████████████████████████| 412.3MB 38kB/s 
     |████████████████████████████████| 411.5MB 44kB/s 
     |████████████████████████████████| 491kB 6.3MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
ERROR: tensorflow-gpu 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.


In [2]:
import tensorflow
print(tensorflow.__version__)

1.15.0


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'   # disable all debugging logs

In [4]:
!git clone https://github.com/zhangabner/gpt-2/
import os
os.chdir('gpt-2')
#Download model weights
# !python download_model.py 117M
# !python download_model.py 345M
# !python download_model.py 774M
!python download_model.py 117M # XL Model
!pip3 -q install -r /content/gpt-2/reqs.txt
#!pip3 -q install -r /content/gpt-2/requirements.txt

Cloning into 'gpt-2'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 336 (delta 4), reused 11 (delta 4), pack-reused 325
Receiving objects: 100% (336/336), 4.68 MiB | 19.46 MiB/s, done.
Resolving deltas: 100% (188/188), done.
Fetching checkpoint: 1.00kit [00:00, 974kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 3.57Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.04Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:02, 8.00Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 5.21Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 1.97Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 1.79Mit/s]                     

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
gamename = "o10r5q5game1.ulx"
file_read = '/content/drive/My Drive/'+gamename+'.play.txt'
file_write = '/content/drive/My Drive/'+gamename+'.story.txt'
fr = open(file_read,"r+") 
raw_text = fr.read() 
raw_text
fw = open(file_write,"w") 
# fw.write(raw_text)
raw_text

'{\'description\': \'-= Cubicle =-\\nWell, here we are in the cubicle. You try to gain information on your surroundings by using a technique you call "looking."\\n\\nYou can see a desk. On the desk you see a folder.\\n\\nThere is a closed portal leading east. There is an exit to the west. Don\\\'t worry, it is unblocked.\\n\\n\\n\', \'inventory\': \'You are carrying:\\n  a loaf of bread\\n\\n\\n\', \'admissible_commands\': [\'drop loaf of bread\', \'eat loaf of bread\', \'examine desk\', \'examine folder\', \'examine loaf of bread\', \'examine portal\', \'go west\', \'inventory\', \'look\', \'open portal\', \'put loaf of bread on desk\', \'take folder from desk\'], \'intermediate_reward\': 1}1: go east -> reward=1, total_reward=1{\'description\': \'-= Cubicle =-\\nWell, here we are in the cubicle. You try to gain information on your surroundings by using a technique you call "looking."\\n\\nYou can see a desk. On the desk you see a folder.\\n\\nThere is an open portal leading east. The

In [ ]:
# read from S3  and save in S3      ai play textgames save as log
# read from S3  and save in S3      gpt2 read play log save as summary
# in comprehend  read summary  save as topic modeling

In [11]:
os.chdir('src')

In [13]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

import model, sample, encoder
model_name='117M'
seed=None
nsamples=3
batch_size=1
length=100
temperature=1
top_k=0
top_p=1
models_dir='/content/gpt-2/models'

models_dir = os.path.expanduser(os.path.expandvars(models_dir))
if batch_size is None:
    batch_size = 1
assert nsamples % batch_size == 0

enc = encoder.get_encoder(model_name, models_dir)
hparams = model.default_hparams()
with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))

if length is None:
    length = hparams.n_ctx // 2
elif length > hparams.n_ctx:
    raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

with tf.Session(graph=tf.Graph()) as sess:
    context = tf.placeholder(tf.int32, [batch_size, None])
    np.random.seed(seed)
    tf.set_random_seed(seed)
    output = sample.sample_sequence(
        hparams=hparams, length=length,
        context=context,
        batch_size=batch_size,
        temperature=temperature, top_k=top_k, top_p=top_p
    )

    saver = tf.train.Saver()
    ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
    saver.restore(sess, ckpt)

    context_tokens = enc.encode(raw_text)
    generated = 0
    for _ in range(nsamples // batch_size):
        out = sess.run(output, feed_dict={
            context: [context_tokens for _ in range(batch_size)]
        })[:, len(context_tokens):]
        for i in range(batch_size):
            generated += 1
            text = enc.decode(out[i])
            print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
            print(text)
            fw.write(text)
    print("=" * 80)




INFO:tensorflow:Restoring parameters from /content/gpt-2/models/117M/model.ckpt
======================================== SAMPLE 1 ========================================
 {' 5'









'Way in: take advanced: 5 go south: take south: take south: take: 5







Take shooter [ala: take squ: 5
k take [make south: take crossover: take: 5
There's face: 6 come down: 5


For the plane: take get Just take going south, take south: just go south

There is got going
======================================== SAMPLE 2 ========================================
} arrival =










for each dish = level .4





for level + level + level now






*


for all

for movement = level + level = total = level 2





for us =






for level
for this level = level




for level +






for half
======================================== SAMPLE 3 ========================================
}1, screen: step: \n, redbird :: return: they simply return us from {'------------------------ end = us











Game = 3
3